In [ ]:
import pandas as pd
import os
import glob
from pathlib import Path

print("🚀 INICIANDO PROCESO DE COMBINACIÓN DE ARCHIVOS CSV (BÚSQUEDA RECURSIVA)")
print("=" * 70)

# Ruta específica del repositorio
repo_url = "https://github.com/forest-scanner/ndvi_zonas_verdes/tree/main/calculos_ndvi"
print(f"📁 Trabajando con: {repo_url}")
print(f"📍 Directorio actual: {os.getcwd()}")

# Búsqueda recursiva de archivos CSV en todas las subcarpetas
print("\n🔍 BUSCANDO ARCHIVOS CSV EN TODAS LAS SUBCARPETAS...")
archivos_csv = glob.glob("**/*.csv", recursive=True)
print(f"📄 Archivos CSV encontrados: {len(archivos_csv)}")

if not archivos_csv:
    print("❌ No se encontraron archivos CSV en ninguna subcarpeta")
    # Mostrar la estructura de carpetas para diagnóstico
    print("\n📂 Estructura de carpetas:")
    for root, dirs, files in os.walk("."):
        level = root.replace(".", "").count(os.sep)
        indent = " " * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        subindent = " " * 2 * (level + 1)
        for file in files:
            if file.endswith(".csv"):
                print(f"{subindent}📄 {file}")
else:
    for archivo in archivos_csv:
        print(f"   ✅ {archivo}")

# Separar archivos en grupos por "tiempo" y "mensual"
print("\n🎯 SEPARANDO ARCHIVOS POR GRUPOS (TIEMPO vs MENSUAL)...")
archivos_tiempo = [archivo for archivo in archivos_csv if 'tiempo' in os.path.basename(archivo).lower()]
archivos_mensual = [archivo for archivo in archivos_csv if 'mensual' in os.path.basename(archivo).lower()]
archivos_otros = [archivo for archivo in archivos_csv if 'tiempo' not in os.path.basename(archivo).lower() and 'mensual' not in os.path.basename(archivo).lower()]

print(f"⏰ Archivos con 'TIEMPO': {len(archivos_tiempo)}")
for archivo in archivos_tiempo:
    print(f"   🔷 {archivo}")

print(f"\n📅 Archivos con 'MENSUAL': {len(archivos_mensual)}")
for archivo in archivos_mensual:
    print(f"   🔶 {archivo}")

if archivos_otros:
    print(f"\n❓ Archivos OTROS (sin 'tiempo' ni 'mensual'): {len(archivos_otros)}")
    for archivo in archivos_otros:
        print(f"   ❔ {archivo}")

# Función mejorada para combinar archivos con información de ruta
def combinar_csv_con_grupo(lista_archivos, nombre_grupo):
    """
    Combina archivos CSV y añade columnas de identificación
    """
    if not lista_archivos:
        print(f"⚠️  No hay archivos en el grupo: {nombre_grupo}")
        return pd.DataFrame()
    
    dataframes = []
    total_filas = 0
    
    print(f"\n🔄 Combinando grupo: {nombre_grupo}")
    print("-" * 50)
    
    for archivo in lista_archivos:
        try:
            # Leer archivo CSV
            df = pd.read_csv(archivo)
            filas_originales = len(df)
            
            # Añadir columnas de identificación
            df['nombre_archivo'] = os.path.basename(archivo)  # Solo el nombre del archivo
            df['ruta_completa'] = archivo  # Ruta completa incluyendo subcarpetas
            df['carpeta'] = os.path.dirname(archivo)  # Carpeta donde está el archivo
            df['grupo'] = nombre_grupo      # Grupo (TIEMPO o MENSUAL)
            df['id_registro'] = range(1, len(df) + 1)  # ID único por registro
            
            dataframes.append(df)
            total_filas += filas_originales
            
            print(f"   ✅ {archivo}")
            print(f"      └─ {filas_originales} filas → Columnas: {len(df.columns)}")
            
        except Exception as e:
            print(f"   ❌ Error en {archivo}: {str(e)}")
    
    # Combinar todos los DataFrames
    if dataframes:
        df_combinado = pd.concat(dataframes, ignore_index=True, sort=False)
        
        # Añadir ID único global
        df_combinado['id_global'] = range(1, len(df_combinado) + 1)
        
        print(f"\n   📊 RESUMEN {nombre_grupo}:")
        print(f"      • Archivos procesados: {len(lista_archivos)}")
        print(f"      • Filas totales: {len(df_combinado):,}")
        print(f"      • Columnas finales: {len(df_combinado.columns)}")
        if not df_combinado.empty and 'carpeta' in df_combinado.columns:
            print(f"      • Carpetas diferentes: {df_combinado['carpeta'].nunique()}")
        
        return df_combinado
    else:
        return pd.DataFrame()

# Procesar grupo TIEMPO
print("\n" + "="*60)
df_tiempo = combinar_csv_con_grupo(archivos_tiempo, "TIEMPO")

# Procesar grupo MENSUAL
print("\n" + "="*60)
df_mensual = combinar_csv_con_grupo(archivos_mensual, "MENSUAL")

# Procesar grupo OTROS (opcional)
if archivos_otros:
    print("\n" + "="*60)
    df_otros = combinar_csv_con_grupo(archivos_otros, "OTROS")

# Mostrar resumen de estructuras
print("\n🔍 ESTRUCTURA DE LOS DATAFRAMES RESULTANTES:")
print("=" * 60)

if not df_tiempo.empty:
    print(f"\n⏰ DATAFRAME TIEMPO:")
    print(f"   • Dimensiones: {df_tiempo.shape[0]} filas × {df_tiempo.shape[1]} columnas")
    print(f"   • Columnas: {list(df_tiempo.columns)}")
    print(f"   • Archivos únicos: {df_tiempo['nombre_archivo'].nunique()}")
    if 'carpeta' in df_tiempo.columns:
        print(f"   • Carpetas únicas: {df_tiempo['carpeta'].nunique()}")
    
    # Mostrar primeras filas
    print(f"\n   👀 Primeras 3 filas:")
    display(df_tiempo.head(3))

if not df_mensual.empty:
    print(f"\n📅 DATAFRAME MENSUAL:")
    print(f"   • Dimensiones: {df_mensual.shape[0]} filas × {df_mensual.shape[1]} columnas")
    print(f"   • Columnas: {list(df_mensual.columns)}")
    print(f"   • Archivos únicos: {df_mensual['nombre_archivo'].nunique()}")
    if 'carpeta' in df_mensual.columns:
        print(f"   • Carpetas únicas: {df_mensual['carpeta'].nunique()}")
    
    # Mostrar primeras filas
    print(f"\n   👀 Primeras 3 filas:")
    display(df_mensual.head(3))

# Guardar resultados
print("\n💾 GUARDANDO ARCHIVOS COMBINADOS...")
print("=" * 60)

if not df_tiempo.empty:
    nombre_archivo_tiempo = "combinado_tiempo.csv"
    df_tiempo.to_csv(nombre_archivo_tiempo, index=False, encoding='utf-8')
    tamaño_kb = os.path.getsize(nombre_archivo_tiempo) / 1024
    print(f"✅ {nombre_archivo_tiempo} - {len(df_tiempo):,} filas - {tamaño_kb:.1f} KB")

if not df_mensual.empty:
    nombre_archivo_mensual = "combinado_mensual.csv"
    df_mensual.to_csv(nombre_archivo_mensual, index=False, encoding='utf-8')
    tamaño_kb = os.path.getsize(nombre_archivo_mensual) / 1024
    print(f"✅ {nombre_archivo_mensual} - {len(df_mensual):,} filas - {tamaño_kb:.1f} KB")

# Archivo combinado total (solo TIEMPO y MENSUAL)
dataframes_para_total = []
if not df_tiempo.empty:
    dataframes_para_total.append(df_tiempo)
if not df_mensual.empty:
    dataframes_para_total.append(df_mensual)

if dataframes_para_total:
    df_total = pd.concat(dataframes_para_total, ignore_index=True, sort=False)
    nombre_archivo_total = "combinado_total.csv"
    df_total.to_csv(nombre_archivo_total, index=False, encoding='utf-8')
    tamaño_kb = os.path.getsize(nombre_archivo_total) / 1024
    print(f"✅ {nombre_archivo_total} - {len(df_total):,} filas - {tamaño_kb:.1f} KB")

# Resumen final
print("\n🎉 PROCESO COMPLETADO - RESUMEN FINAL")
print("=" * 70)

# Calcular carpetas diferentes de forma segura
carpetas_unicas = set()
if not df_tiempo.empty and 'carpeta' in df_tiempo.columns:
    carpetas_unicas.update(df_tiempo['carpeta'].unique())
if not df_mensual.empty and 'carpeta' in df_mensual.columns:
    carpetas_unicas.update(df_mensual['carpeta'].unique())

if not df_tiempo.empty:
    archivos_tiempo_list = df_tiempo['nombre_archivo'].unique()
    if 'carpeta' in df_tiempo.columns:
        carpetas_tiempo = df_tiempo['carpeta'].unique()
    print(f"\n⏰ GRUPO TIEMPO:")
    print(f"   • Archivos: {len(archivos_tiempo_list)}")
    if 'carpeta' in df_tiempo.columns:
        print(f"   • Carpetas: {len(carpetas_tiempo)}")
    print(f"   • Filas totales: {len(df_tiempo):,}")

if not df_mensual.empty:
    archivos_mensual_list = df_mensual['nombre_archivo'].unique()
    if 'carpeta' in df_mensual.columns:
        carpetas_mensual = df_mensual['carpeta'].unique()
    print(f"\n📅 GRUPO MENSUAL:")
    print(f"   • Archivos: {len(archivos_mensual_list)}")
    if 'carpeta' in df_mensual.columns:
        print(f"   • Carpetas: {len(carpetas_mensual)}")
    print(f"   • Filas totales: {len(df_mensual):,}")

print(f"\n📊 TOTAL GENERAL:")
print(f"   • Archivos CSV encontrados: {len(archivos_csv)}")
print(f"   • Filas combinadas totales: {len(df_total) if 'df_total' in locals() else 0:,}")
print(f"   • Carpetas diferentes: {len(carpetas_unicas)}")

print(f"\n📍 Archivos guardados en: {os.getcwd()}")
